# Projet Machine Learning

Mélissa Zennaf - Janvier 2021

Les notebooks suivants ainsi que la librairie qui leur est associée constituent le fruit du Projet Machine Learning correspondant à l'enseignement du même nom au sein du Master Mécen. 

L'objectif de cette étude sera, dans un premier temps, de mettre en place une problématique , et de modéliser cette problématique. Dans un second temps, on s'intéressera aux données disponibles sur internet pour répondre à celle-ci et on effectuera du web scraping afin de les récupérer de manière automatisée. La phase suivante sera une phase de preprocessing et de nettoyage des données afin de les rendre exploitables par des algorithmes de machine learning. Pour finir, on définira certains modèles pertinents pour répondre au mieux à la problématique et on les mettra en oeuvre. On essayera de distinguer le meilleur modèle parmis ceux testés et de présenter ses résultats afin de répondre à la problématique.

# Problématique

En tant que future jeune diplômée, je serai prochainement amenée à trouver du travail et ce sera certainement dans un autre départemet que l'Indre-et-Loire. Ma problématique est de savoir dans quel département français je suis susceptible de percevoir la rémunération la plus élevée. Existe-t-il des disparités de salaire entre les départements français ? Quels sont les déterminents et aboutissants de ces disparités ? Dans quel département s'installer pour maximiser sa rémunération ?

Pour tenter de répondre à cette problématique, je vais m'appuyer sur des données économiques, géographiques, démographiques et financières concernant les départements français. 

A première vue, le salaire (ou plus précisément le salaire moyen) est déterminé par de multiples conditions. Les plus évidentes sont tout d'abord le niveau de vie et le dynamisme économique du département en question. On peut également citer des déterminents démographiques comme la population, ou des déterminents financiers comme le taux d'imposition. 

Comme le revenu moyen est une variable quantitative et continue, on s'attend à mobiliser des techniques de régression. Au niveau mathématique, on peut donc définir cela comme la recherche des $X$ pour lesquels une relation $y=f(X)$ est possible et pertinente avec $y$ la variable à expliquer, à savoir le revenu moyen au sein d'un département. On s'attachera à définir la relation $f()$ dans la partie trois de cette étude.

Pour la réalisation technique de ce projet, on utilisera le langage python. La plupart des fonctions seront stockées dans un script annexe. Les librairies utiles seront numpy, requests, bs4, pandas, string, seaborn, et scikitLearn.

Nous commençons par une phase de recherche internet et de récupération des données.

# Web scraping

**Dans cette première partie, nous allons scraper des données sur le web afin des se constituer une base de données.**

On commence par importer les librairies et fonctions nécessaires à ce travail.

In [1]:
from requests import get
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
from string import digits

In [2]:
from script import adresse, get_pages, find_jdn, find_wiki, find_wiki_2, gestion_ligne5, gestion_ligne9, gestion_ligne12, gestion_ligne14, gestion_ligne6, gestion_ligne7, gestion_ligne8, construit_df 

Au cours de la phase de modélisation, nous avons donc établi que nous cherchons à extraire des données sur les départements français. Pour ceci, on s'est constitué un catalogue des sources internet. Les liens menant à ces pages sont les suivants :

* [pib des départements français - Wikipédia](https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_produit_int%C3%A9rieur_brut_par_habitant)

* [population et superficie des départements français - Wikipédia](https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_population_et_superficie)

* [nombre de communes par département - Wikipédia](https://fr.wikipedia.org/wiki/Nombre_de_communes_par_d%C3%A9partement_en_France_au_1er_janvier_2014)

* [salaire par département - Journal du net](http://www.journaldunet.com/business/salaire/classement/departements/salaires)

* [revenu déclaré par département - Journal du net](http://www.journaldunet.com/economie/impots/classement/departements/revenu-fiscal)

* [taxe foncière par département - Journal du net](http://www.journaldunet.com/economie/impots/classement/departements/taxe-fonciere-bati)

* [impôt sur le revenu par département - Journal du net](http://www.journaldunet.com/economie/impots/classement/departements/impot-revenu)

Le principe est de récupérer les données issues de ces sites internet via des fonction indépendantes qui récupérent les colonnes de chaque tableau pour chaque ligne du tableau. On obtient finalement une liste de liste où une liste correspond aux données pour une variable. Pour chaque source, nous avons donc une liste de liste. 

On utilise requests, en particulier la commande ``get`` puis on utilise BeatifulSoup ainsi que le parceur "lxml". 

## Scraping des données 

Pour chaque source, nous faisons appel à la fonction construite par nos soins. On stock le résultat sous la forme d'une liste de liste. Par exemple, pour la première source, la fonction est construite de la sorte :

``def adresse(ad):
    page_brute=get(ad)
    texte = page_brute.text
    texte = texte.replace("\n", " ")
    texte = texte.replace("\t", " ")
    motif = re.compile("<tr>\s*<td><a(.*?)</tr>")
    soupe = BS(texte, "lxml")
    return soupe``
   
Cette fonction fait appel à la library requests et nous permet d'obtenir le code html des pages. Une fois le texte obtenu, on applique une deuxième fonction adaptée à la source (Wikipédia ou Journal du net) qui nous permet de trouver les balises correspondantes à une ligne du tableau dont on veut récupérer les données. Par exemple, pour une source Journal du net, on cherche les balises html tr qui correspondent à une ligne. 

``def find_jdn(soupe):
    balise_table=soupe.find_all(name="table", attrs={"class" : ["odTable"]})
    table=balise_table[0]
    corps=table.find_next(name="tbody")
    lignes=corps.find_all(name="tr")
    ligne1,*_ = lignes
    return lignes``
    
On obtient donc le texte du corps du tableau. Une fois cela obtenu, on cherche à récupérer les données de chaque ligne grâce à la fonction suivante :

``def gestion_ligne5(lignes):
    for i in range(len(lignes)):
        colonnes = lignes[i].find_all(name="td")
        rang, nom, salaire = colonnes
        s=nom.text.replace('(','').replace(')', '')
        remove_digits = str.maketrans('', '', digits)
        res = s.translate(remove_digits)
        liste_ligne_nom.append(
        res.replace(' ', '').replace('', ' '))
        liste_ligne_code1.append(int(re.findall('\d+', nom.text)[0]))
        liste_ligne_rmoyen.append(
        salaire.text.rstrip().replace("\xa0", "").replace(" ", "").replace("€nets/mois", ""))
    return[liste_ligne_nom, liste_ligne_rmoyen, liste_ligne_code1]``
    
Comme expliqué, nous récupérons une liste par variable dans le tableau de données. 

Toutes ces fonctions sont appliquées de manière successive. On commence toujours par une ligne de code telle que celle-ci : 

``lignes3=list(find_wiki(adresse("lien url")))``

Pour la bonne exécution du programme, l'output doit être une liste. Puis, on applique notre dernière fonction :

``res3=gestion_ligne6(lignes3)``

Dans le cas le plus simple, toutes les données sont sur la même page, dans ce cas, on met en entrée le lien url et on récupère directement les données. 

In [3]:
lignes3=list(find_wiki(adresse("https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_produit_int%C3%A9rieur_brut_par_habitant")))
res3=gestion_ligne6(lignes3)

In [4]:
lignes4=list(find_wiki_2(adresse("https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_population_et_superficie")))
del lignes4[3]
res4=gestion_ligne7(lignes4)

In [5]:
lignes5=list(find_wiki_2(adresse("https://fr.wikipedia.org/wiki/Nombre_de_communes_par_d%C3%A9partement_en_France_au_1er_janvier_2014")))
res5=gestion_ligne8(lignes5)

Dans ce cas un peu différent, les données d'un tableau apparaissent sur deux pages ayant un url différent. Le premier se termine par "/salaire" et celui de la deuxième page se termine par "/salaire?page=2". Il est assez compliqué de créer une boucle dans ce cas donc on choisit d'appeler la fonction à deux reprises.

In [6]:
lignes=list(find_jdn(adresse("http://www.journaldunet.com/business/salaire/classement/departements/salaires")))
lignes2=list(find_jdn(adresse("http://www.journaldunet.com/business/salaire/classement/departements/salaires?page=2")))
gestion_ligne5(lignes)
res2=gestion_ligne5(lignes2)

In [7]:
lignes4=list(find_jdn(adresse("http://www.journaldunet.com/economie/impots/classement/departements/revenu-fiscal")))
lignes42=list(find_jdn(adresse("http://www.journaldunet.com/economie/impots/classement/departements/revenu-fiscal?page=2")))
gestion_ligne9(lignes4)
res1=gestion_ligne9(lignes42)

In [8]:
lignes10=list(find_jdn(adresse("http://www.journaldunet.com/economie/impots/classement/departements/taxe-fonciere-bati")))
lignes102=list(find_jdn(adresse("http://www.journaldunet.com/economie/impots/classement/departements/taxe-fonciere-bati?page=2")))
gestion_ligne14(lignes10)
res6=gestion_ligne14(lignes102)

Enfin, dans ce dernier cas, il y a trois pages dont les liens se terminent de la même manière que précedemment mais avec "/salaire?page=3" en plus. On va cette fois créer une boucle qui nous permet de créer des liens pour les pages 2 et 3. Cette fonction est codée de la manière suivante :

``def get_pages(token, nb):
    pages = []
    for i in range(2,nb+1):
        j = token + str(i)
        pages.append(j)
    return pages``
    
Evidemment pour seulement 2 pages, on aurait pu les créer manuellement mais cette fonction peut être généralisée à un plus grand nombre de page, d'où son intérêt. Dans cette fonction, token est le lien et nb le nombre de déclinaison que l'on souhaite. Comme il n'existe pas de pages avec "/salaire?page=1", la liste va de 2 à 3.

Une fois les url créés, on applique une boucle qui scrape les données pour ces deux liens. 

In [9]:
lignes8=list(find_jdn(adresse("http://www.journaldunet.com/economie/impots/classement/departements/impot-revenu")))
gestion_ligne12(lignes8)
token = 'http://www.journaldunet.com/economie/impots/classement/departements/impot-revenu?page='
pages = get_pages(token,3)
for i in range(len(pages)) :
    lignes82=list(find_jdn(adresse(pages[i])))
    res8=gestion_ligne12(lignes82)

## Constitution de la base de données

Nous avons donc en notre possession les listes de liste contenant nos données. On va à présent les transformer en matrice dans le but de les enregistrer en tant que data frame. On commence par créer les indices de colonne pour chaque data frame.

On transforme les outputs de notre scraping en matrices puis en data frame en faisant appel à la librairie pandas. On utilise ensuite plusieurs fois la fonction ``pandas.merge()`` afin de réunir nos data frame en un seul. On précise la colonne qui permet la réunion, elle peut être la colonne correspondant au nom ou au code du département selon les cas. 

In [10]:
df=construit_df(res1, res2, res3, res4, res5, res6, res8)

Nous obtenons le data frame suivant. On peut voir que l'on a plus que 91 lignes sur les 102 départements français. Cela est dû au fait que lors de la réunification de nos data frame, certaines données dont le critère de réunion n'était pas présent dans le data frame précédent ont été abandonnées. Cela explique que nous perdions des données. 

In [13]:
df.head()

,code_x,nom,pop_2012,pop_2014,pop_2016,pop_2017,pop_2018,superficie,densite,nb_commune,...,hab_m,pib_2015,pib_2005,pib_2000,décla,code_y,rmoyen,code,taxe_om,impot
0,59,N o r d,2587128,2603472,2603723,2604361,2604361,"5742,8","453,8",650,...,3968,,23567,19794,2427,59,24887,59,"18,07",3414
1,75,P a r i s,2240621,2220445,2190327,2187526,2187526,"105,4","20720,0",1,...,2249975,96400,75439,67502,4007,75,48301,75,"13,50",11750
2,13,B o u c h e s - d u - R h ô n e,1984784,2006069,2019717,2024162,2024162,5087,"399,9",119,...,16604,34200,27818,23521,2570,13,26751,13,"21,06",3996
3,93,S e i n e - S a i n t - D e n i s,1538726,1571028,1606660,1623111,1623111,"236,2","6918,1",40,...,38248,38900,27420,23305,2398,93,21768,93,"21,91",3095
4,92,H a u t s - d e - S e i n e,1586434,1597770,1603268,1609306,1609306,"175,6","9199,6",36,...,43934,97500,73277,62244,3885,92,45966,92,"16,04",9162


On enregistre finalement ce data frame au dormat csv en prévision de la suite de nos analyses. 

In [12]:
filename = 'df.csv'
df.to_csv(filename, index=False, encoding='utf-8')

Au cours de cette première phase du projet, on a commencé par poser la problématique et élaborer un catalogue de sources disponibles pour récupérer des données. On a ensuite procédé au scraping de celles-ci et à leur transformation en un seul jeu de données exploitable. Nous allons poursuivre sur la phase de nettoyage et de préparation des données. 